In [97]:
import pandas as pd
import numpy as np
from typing import List, Tuple

In [98]:
# Parameters
spot_filename:str = "/tmp/index.csv"
options_filename:str = "../../data/nifty_options_eod.h5"
start_day:int = 4
holding_days:int = 7
de:int = 0
step:int = 50 
options:Tuple[Tuple[str, float]] = (
    ('p', 0),
    ('p', 0.02),
)

In [99]:
index = pd.read_csv(spot_filename, parse_dates=['date'])
opt = pd.read_hdf(options_filename).sort_values(by=['date'])
rename = {
    'open_price': 'open',
    'high_price': 'high',
    'low_price': 'low',
    'close_pric': 'close'
}
opt = opt.rename(columns=rename)
opt['de'] = (opt['expiry_date'] - opt['date']).dt.days

In [100]:
ce = opt.query("opt_type=='CE'")
pe = opt.query("opt_type=='PE'")

In [101]:
def get_result(options_data, spot_data, opt='p', wkday=4, de=0, itm=0, step=100, holding_days=7):
    weekly = spot_data.query(f"wkday=={wkday}").copy()
    if opt.lower() == 'p':
        m = 1-itm
    else:
        m = 1+itm
    weekly['strike'] = [int((x*m)/step)*step for x in weekly.close.values]
    if opt.lower() == 'p':
        weekly['strike'] = weekly['strike'] + step
    opts = options_data.merge(weekly[['date', 'close', 'strike']].rename(
        columns={'close':'spot'}), on=['date', 'strike'])
    opts = opts.query(f"de>{de}")
    opts = opts.sort_values(by='de').groupby('date').first().reset_index()
    cols = ['date', 'contract_d', 'strike', 'expiry_date', 'close', 'spot']
    entries = opts[cols].copy()
    entries['entry_date'] = entries.date.copy()
    exit_frame = entries.copy()
    exit_frame['date'] = exit_frame.date + pd.DateOffset(days=holding_days)
    del exit_frame['close'] # Since we need the other close in expiry
    del exit_frame['contract_d'] 
    del exit_frame['spot']
    opt2 = exit_frame.merge(options_data, on=['date', 'strike', 'expiry_date'])
    opt2 = opt2.merge(index[['date', 'close']].rename(
        columns={'close': 'spot'}), on=['date'])
    exits = opt2[cols + ['entry_date']]
    exits = exits.rename(columns={'date':'exit_date', 'entry_date': 'date'})
    trades = entries.merge(exits, on=['strike', 'expiry_date', 'contract_d', 'date'])
    rename = {
        'date_x': 'entry_date',
        'close_x': 'entry_price',
        'spot_x': 'entry_spot',
        'date_y': 'exit_date',
        'close_y': 'exit_price',
        'spot_y': 'exit_spot'
    }
    trades = trades.rename(columns=rename)
    trades['pnl'] = trades.eval('exit_price-entry_price')
    trades['de'] = (trades['expiry_date'] - trades['entry_date']).dt.days
    return trades


In [102]:
collect = []
for i,(opt,strike) in enumerate(options):
    data = ce if opt =='c' else pe
    temp = get_result(data, index, itm=strike,step=step, wkday=start_day, 
                      holding_days=holding_days)
    temp['name'] = f"opt{i}"
    collect.append(temp)
res = pd.concat(collect)
del collect
res.pnl.describe()

count     508.000000
mean      -11.586713
std       164.067914
min      -496.150000
25%       -77.037500
50%       -43.375000
75%         4.200000
max      1564.200000
Name: pnl, dtype: float64

In [103]:
res2 = res.pivot(index='date', columns=['name'], values='pnl')

In [110]:
mx = index.set_index('date').merge(res2, right_index=True, left_index=True)
mx['pnl'] = mx.eval('-opt0+opt1')
mx.query('close>0').loc['2019':, 'opt0':].describe()

,opt0,opt1,pnl
count,119.000000,119.000000,119.000000
mean,-17.121008,-12.967647,4.153361
std,238.417189,210.342909,53.134555
min,-496.150000,-451.950000,-102.700000
25%,-124.725000,-81.050000,-54.750000
50%,-83.700000,-58.250000,29.900000
75%,3.900000,-27.925000,45.625000
max,1564.200000,1504.800000,69.350000
